# Importing the Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

output_path = "/content/drive/MyDrive/266_Project/test_df_subset.csv"
test_df_subset = pd.read_csv(output_path)

# Treatment Model: LLAMA for Debiasing the Sentences

* Use LLaMA to Debias Sentences: Debias all sentences in both the training and test sets using LLaMA.
* Feed Debiased Sentences into the RoBERTA Classifier (the best performing classification treatment model)
* Evaluate Classification Output: Check if the classifier’s predictions are now more neutral (i.e., classified as "center") for previously biased sentences, which would indicate effective debiasing.


### 1. Set Up LLaMA Model for Debiasing (merge with fine-tuned model)

In [3]:
!pip install transformers accelerate huggingface-hub

In [35]:
from huggingface_hub import login
login(token='') #add hugging face token

Import the fine tuned LLAMA2 model and merge with the LLAMA2 base.

In [5]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.9 MB/s eta 0:00:00


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "meta-llama/Llama-2-7b-hf"  # Base model
lora_weights_path = "/content/drive/MyDrive/266_Project/4_LLAMA_PEFT_QLORA/qlora_debiased_model"  # LoRA weights

lora_tokenizer = AutoTokenizer.from_pretrained(lora_weights_path)

base_tokenizer = AutoTokenizer.from_pretrained(model_name)

base_tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add padding token 

if len(lora_tokenizer) > len(base_tokenizer):
    base_tokenizer.resize_token_embeddings(len(lora_tokenizer))

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False
)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.resize_token_embeddings(len(lora_tokenizer))

# Load LoRA weights into the base model
final_model = PeftModel.from_pretrained(base_model, lora_weights_path)

# Merge LoRA weights into the base model
final_model = final_model.merge_and_unload()

final_model_path = "/content/drive/MyDrive/266_Project/5_FINAL_LLAMA_DEBIASING/final_debiased_model"
final_model.save_pretrained(final_model_path)
base_tokenizer.save_pretrained(final_model_path)

print(f"Final model saved to {final_model_path}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Final model saved to /content/drive/MyDrive/266_Project/5_FINAL_LLAMA_DEBIASING/final_debiased_model


In [7]:
pipe = pipeline(
    "text-generation",
    model="/content/drive/MyDrive/266_Project/5_FINAL_LLAMA_DEBIASING/final_debiased_model",
    tokenizer="/content/drive/MyDrive/266_Project/5_FINAL_LLAMA_DEBIASING/final_debiased_model")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [8]:
# sample debiased sentence
sentence = "The government is flooding our country with immigrants who don't share our values and are causing crime rates to soar."
result = pipe(
    f"Rewrite the following sentence to be neutral and unbiased:\n\n{sentence}\n\nRewritten sentence:",
    max_length=100,  
    do_sample=True,  
    temperature=0.7,  
    top_k=50, 
    top_p=0.9,  
    repetition_penalty=1.2, 
    num_return_sequences=1 
)
print(result[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Rewrite the following sentence to be neutral and unbiased:

The government is flooding our country with immigrants who don't share our values and are causing crime rates to soar.

Rewritten sentence: The number of people arriving in this nation is increasing, which has caused a rise in crime statistics.


In [9]:
def debias_sentence(sentence):
    prompt = f"Rewrite the following sentence to have neutral sentiment, remove any political bias while maintaining its original meaning. The rewritten sentence should be neutral and fact-based:\n\n{sentence}\n\nRewritten sentence:"
    inputs = base_tokenizer(prompt, return_tensors="pt").to(device)

    final_model.config.pad_token_id = final_model.config.eos_token_id

    outputs = final_model.generate(
        **inputs,
        max_length=150,  
        num_return_sequences=1,  
        do_sample=True,  
        temperature=0.25,
        top_k=50,  # 
        top_p=0.9,  
        repetition_penalty=1.2  
    )

    debiased_sentence = base_tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Rewritten sentence:" in debiased_sentence:
        debiased_sentence = debiased_sentence.split("Rewritten sentence:")[-1].strip()

    if "\n\n" in debiased_sentence:
        debiased_sentence = debiased_sentence.split("\n\n")[0].strip()

    return debiased_sentence

### 2. Debias Sentences Using LLaMA

In [10]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [11]:
test_df_subset['debiased_text'] = test_df_subset['text'].apply(debias_sentence)

In [12]:
pd.set_option('display.max_colwidth', None)
print(test_df_subset[['text', 'debiased_text']].head())

                                                                                                                                                                                                                                                                                                                                                                              text  \
0                                                                                       Indeed, that original NYT report suggested that the management at Trump’s club in New Jersey was well aware of the fact that it employed undocumented immigrants, and it deliberately took steps to obscure the fact that it broke the law – repeatedly, over the course of several years.   
1                                                                                      How we treat people at or inside our border certainly deserves attention, but we cannot ignore that many people come to the United States in the first place because 

In [13]:
output_path = "/content/drive/MyDrive/266_Project/debiased_df.csv"
test_df_subset.to_csv(output_path, index=False)